In [2]:
from holisticai.benchmark import BiasMitigationBenchmark

# select the benchmark task type and the bias mitigation type
benchmark = BiasMitigationBenchmark("regression", "inprocessing")

# get the datasets and mitigators
datasets = benchmark.get_datasets()
mitigators = benchmark.get_mitigators()

# visusalize the current benchmark result for the selected task and mitigation type
results = benchmark._build_benchmark(datasets=datasets, mitigators=mitigators)

us_crime dataset


/home/kcosta/repos/hai/holisticai/src/holisticai/bias/mitigation/inprocessing/commons/regression/_constraints.py:76: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  pos_basis[i][group] = 1
/home/kcosta/repos/hai/holisticai/src/holisticai/bias/

In [3]:
class MyMitigator():
    pass

my_mitigator = MyMitigator()
benchmark.run(my_mitigator)